# Hausarbeit im Seminar Textanalyse WiSe 22/23

Zunächst werden die Parteien definieiert und die Wahlprogramme (soweit vorhanden) eingelsen.

In [2]:
# import modules
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import GermanStemmer



In [3]:
PARTIES = [
    "ALL_BB",
    "GDF",
    "GHG",
    "JUSO",
    "LHG",
    "LISTE_VOLT",
    "NORDCAMPUS",
    "RCDS"
]

PROGRAMS = {}

for party in PARTIES:
    with open(f"{party}.txt", "r", encoding="utf-8") as f:
        file_content = f.read()
        PROGRAMS[party] = file_content

df = pd.DataFrame(PROGRAMS.items(), columns=["party", "program"] )

df

,party,program
0,ALL_BB,"test test test Email sent, waiting for response"
1,GDF,Hochschulpolitik - Nicht mehr und nicht wenige...
2,GHG,"Liebe Studis, \nEs ist wieder so weit, die Hoc..."
3,JUSO,"Liebe Wähler*innen,\nvor euch seht ihr unser W..."
4,LHG,AStA-Beiträge senken\nAllgemeine Studiengebühr...
5,LISTE_VOLT,"Volt & Die LISTE verstehen sich als linkes, pr..."
6,NORDCAMPUS,Mehr interdisziplinäre Zusammenarbeit\nViele S...
7,RCDS,Auslandsstudium\nWer ein Semester im Ausland v...


## Data Cleaning
Im nächsten Schritt werden die Wahlprogramme gereinigt. Das Bedeutet, dass einige, für die Analyse unwichtige oder störende, Elemente gelöscht werden. Darunter zählen:
- Zeilenumbrüche
- großbuchstaben (werden klein geschrieben)
- anderes

In [4]:
TO_REPLACE = {
    "&": "und",
    "*": "_"
}

# Alle Buchstaben klein schreiben
df.program = df.program.str.lower()

for e in TO_REPLACE:
    df.program = df.program.str.replace(e, TO_REPLACE[e])


# tokenize
for i, prog in df.program.items():
    df.program[i] = word_tokenize(prog, language="german")
# rerun cell above to fix data type if you get an error

df.program

C:\Users\User\AppData\Local\Temp\ipykernel_1564\2861057112.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.program = df.program.str.replace(e, TO_REPLACE[e])


0    [test, test, test, email, sent, ,, waiting, fo...
1    [hochschulpolitik, -, nicht, mehr, und, nicht,...
2    [liebe, studis, ,, es, ist, wieder, so, weit, ...
3    [liebe, wähler_innen, ,, vor, euch, seht, ihr,...
4    [asta-beiträge, senken, allgemeine, studiengeb...
5    [volt, und, die, liste, verstehen, sich, als, ...
6    [mehr, interdisziplinäre, zusammenarbeit, viel...
7    [auslandsstudium, wer, ein, semester, im, ausl...
Name: program, dtype: object

In [5]:
# stopwords definieren
custom_stopwords = ["test"]
stopwords_used = stopwords.words("german") + custom_stopwords

# stemming setup
stemmer = GermanStemmer()


for i, prog in df.program.items():
    # remove stopwords
    no_stopwords = [word for word in prog if word not in stopwords_used]

    # remove short tokens
    new_list = [word for word in no_stopwords if len(word) > 1]

    # stemming
    new_list = [stemmer.stem(word) for word in new_list]
    # TODO studierende wird nicht zu studi 

    # save to df
    df.program[i] = new_list




df.program

0                 [email, sent, waiting, for, respons]
1    [hochschulpolit, mehr, wenig, hochschulpolit, ...
2    [lieb, studis, weit, hochschulwahl, steh, 16.,...
3    [lieb, wahler_inn, seht, wahlprogramm, hochsch...
4    [asta-beitrag, senk, allgemein, studiengebuhr,...
5    [volt, list, versteh, link, progressiv, queer-...
6    [mehr, interdisziplinar, zusammenarbeit, viel,...
7    [auslandsstudium, wer, sem, ausland, verbringt...
Name: program, dtype: object

In [11]:
def create_word_frequency_matrix(index: int) -> pd.DataFrame:
    absolute_freq = pd.Series(df.program[index]).value_counts()
    relative_freq = pd.Series(df.program[index]).value_counts(normalize=True)
    word_freq_df = pd.DataFrame({
        # "word": absolute_freq.keys(),
        "abs_freq": absolute_freq,
        "rel_freq": relative_freq
    })

    return(word_freq_df)

#TODO fix stemming/lemmatization (ask)
#TODO think about research question (lol)


ALL_BB
         abs_freq  rel_freq
email           1       0.2
sent            1       0.2
waiting         1       0.2
for             1       0.2
respons         1       0.2

GDF
         abs_freq  rel_freq
studier        19  0.022432
digital         9  0.010626
student         9  0.010626
studium         9  0.010626
ford            8  0.009445
...           ...       ...
verbess         1  0.001181
schutz          1  0.001181
verbind         1  0.001181
beweg           1  0.001181
sieht           1  0.001181

[549 rows x 2 columns]

GHG
                abs_freq  rel_freq
studier               15  0.012909
uni                   14  0.012048
mehr                  13  0.011188
universitat           13  0.011188
student               12  0.010327
...                  ...       ...
manifestiert           1  0.000861
1.                     1  0.000861
generation             1  0.000861
herausfordernd         1  0.000861
wahlt                  1  0.000861

[749 rows x 2 columns]

JUSO
    